# extended_data_fig5a

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import anndata
from matplotlib import rcParams
sc.set_figure_params(color_map='viridis')
sc.settings.verbosity = 2

In [ ]:
%pylab inline
import warnings
warnings.filterwarnings("ignore")
from gssnng import score_cells
from glob import iglob
from tqdm import tqdm

In [ ]:
fig4b=sc.read("Fig4b.h5")

In [ ]:
figsize(4,4)
sc.pl.umap(fig4b, color = ["SPP1","MOG","MBP","MAG"], frameon = False,ncols=2,vmax=3,save="extended_data_fig5a.pdf")

# extended_data_fig6a

In [ ]:
fig5a=sc.read("Fig5a.h5")

In [ ]:
#neurotransmitter_receptor_relate.gmt file consists of neurotransmitter receptor genes
score_cells.with_gene_sets(adata=fig5a,                         # AnnData object
                            gene_set_file='neurotransmitter_receptor_relate.gmt', # File path of gene sets
                            groupby='region',                  # Will sample neighbors within this group
                            smooth_mode='connectivity',         # Smooths matrix using distance weights from NN graph.
                            recompute_neighbors=32,             # Rebuild nearest neighbor graph with groups, 0 turns off function
                            score_method='singscore',           # Method of scoring
                            method_params={'normalization':'theoretical'},  # Special parameters for some methods 
                            ranked=True,                        # Use ranked data, True or False
                            cores=36)
sc.pl.matrixplot(fig5a,['Glutamate', 'Glycine', 'Acetylcholine', 'Epinephrine and Norepinephrine', '5-HT', 'GABA'],
                 groupby='region',cmap='RdBu_r',save='extended_data_fig6a.pdf')

# extended_data_fig6b

In [ ]:
from adpbulk import ADPBulk
fig5a=sc.read("Fig5a.h5")
# initialize the object
adpb = ADPBulk(fig5a, ["donor_status", "region"])

# perform the pseudobulking
pseudobulk_matrix = adpb.fit_transform()

# retrieve the sample meta data (useful for easy incorporation with edgeR)
sample_meta = adpb.get_meta()

print(sample_meta)
print(pseudobulk_matrix)
import pandas as pd

transposed_matrix = pseudobulk_matrix.transpose()
print(transposed_matrix)
#generate pseudobulk matrix
transposed_matrix.to_csv('neurotransmitter_pseudobulk.csv', index=True)  

In [ ]:
library(pheatmap)
library(dplyr)
library(RColorBrewer)
rm(list=ls())
gc()
#########Left
pseudobulk_matrix <- read.csv("neurotransmitter_pseudobulk.csv", header = TRUE, sep=",",row.names=1)
colnames(pseudobulk_matrix)<-c("AD_Prefrontal cortex","Healthy_Hippocampus","Healthy_Prefrontal cortex","RRMS_Prefrontal cortex","SPMS_Prefrontal cortex","TLE_Hippocampus")
desired_order <- c("Healthy_Hippocampus", "TLE_Hippocampus", "Healthy_Prefrontal cortex", "AD_Prefrontal cortex", "RRMS_Prefrontal cortex", "SPMS_Prefrontal cortex")
pseudobulk_matrix <- pseudobulk_matrix[, desired_order]

group_list <- factor(c("Hippocampus","Hippocampus","Prefrontal cortex","Prefrontal cortex","Prefrontal cortex","Prefrontal cortex"))
neu<-read.table("neurotransmitter.txt",header=T,sep="\t",row.names=1)
matching_genes <- intersect(rownames(pseudobulk_matrix), rownames(neu))

input_data <- pseudobulk_matrix[rownames(neu), ]

dat=log10(edgeR::cpm(input_data)+1)
table(group_list)
n=t(scale(t(dat))) 
n <- na.omit(n)  

n[n>2]=2
n[n< -2]= -2

ac=data.frame(Region=group_list)
rownames(ac)=colnames(n) 
ac$Donor_status<-c("Healthy","TLE","Healthy","AD","RRMS","SPMS")
library(dplyr)

neu_row=
ann_colors = list(
  Region = c(Hippocampus = "#74CAFF", `Prefrontal cortex` = "#B3138A"),
  Donor_status = c(Healthy = "#E68700", TLE="#9667BF", AD = "#5D6AB3", RRMS = "#52BEA5", SPMS = "#9BCA44"),
  Receptor_type = c(
    Glutamate = "#B89582", Glycine = "#EECBC8", Acetylcholine = "#D68985",
    `Epinephrine and Norepinephrine` = "#DEA05B", `5-HT` = "#363752", GABA = "#74717E"
  )
)

p1=pheatmap(n, 
         scale = "none",
         color =colorRampPalette(c("#3d4c9e","grey", "#eee814"))(100), show_rownames = T,show_colnames = F,cellwidth = 10, cellheight = 10,
         cluster_row = F, cluster_col = FALSE, annotation_col=ac, annotation_row=neu,annotation_colors = ann_colors)


p1


library(ggplot2)
ggsave(filename="extended_data_fig6b_left.pdf",plot=p1, width = 6, height = 18)



##########Right
library(pheatmap)
library(dplyr)
library(RColorBrewer)

pseudobulk_matrix <- read.table("neurotransmitter_pseudobulk.csv", header = TRUE, sep=",")


pha<-read.table("pfc_gene_anno.txt",header=T,sep="\t",row.names=1)
group_list <- factor(c("Hippocampus","Hippocampus","Prefrontal cortex","Prefrontal cortex","Prefrontal cortex","Prefrontal cortex"))

matching_genes <- intersect(rownames(pseudobulk_matrix), rownames(pha))

pha_df <- pseudobulk_matrix[matching_genes, ]
pha_df <- pha_df[rownames(pha), ]
dat=log10(edgeR::cpm(pha_df)+1)
table(group_list)
n=t(scale(t(dat))) 
n <- na.omit(n)  

n[n>2]=2
n[n< -2]= -2

#pheatmap(n,show_colnames =F,show_rownames = F)
ac=data.frame(Region=group_list)
rownames(ac)=colnames(n) 
ac$Donor_status<-c("Healthy","TLE","Healthy","AD","RRMS","SPMS")
library(dplyr)

neu_row=
ann_colors = list(
  Region = c(Hippocampus = "#74CAFF", `Prefrontal cortex` = "#B3138A"),
  Donor_status = c(Healthy = "#E68700", TLE="#9667BF", AD = "#5D6AB3", RRMS = "#52BEA5", SPMS = "#9BCA44"),
  Type = c(
    phagocytosis = "#7470B3", Chemokine= "#FE9389", Cytokine = "#FF80DE"
  )
)

p2=pheatmap(n, 
         scale = "none",
         color =colorRampPalette(c("#3d4c9e","grey", "#eee814"))(100), show_rownames = T,show_colnames = F,cellwidth = 10, cellheight = 10,
         cluster_row = F, cluster_col = FALSE, annotation_col=ac, annotation_row=pha,annotation_colors = ann_colors)


p2
ggsave(filename="extended_data_fig6b_right.pdf",plot=p2, width = 6, height = 18)
p1+p2


# extended_data_fig6c

In [ ]:
from adpbulk import ADPBulk
fig5a=sc.read("Fig5a.h5")
# initialize the object
adpb = ADPBulk(fig5a, ["region", "donor_ID"])

# perform the pseudobulking
pseudobulk_matrix = adpb.fit_transform()

# retrieve the sample meta data (useful for easy incorporation with edgeR)
sample_meta = adpb.get_meta()

print(sample_meta)
print(pseudobulk_matrix)
import pandas as pd

transposed_matrix = pseudobulk_matrix.transpose()
print(transposed_matrix)
#generate pseudobulk matrix
transposed_matrix.to_csv('fig5_region_donor_ID_pseudobulk.csv', index=True)  

# Pearson correlation analysis between TNF (left) and IL1A (right) with glutamatergic receptors

## TNF

In [ ]:
library(tidyverse)
library(ggplot2)
library(ggpubr)
library(cowplot)
library(ggExtra)
library(ggplot2)
library(cols4all)
pseudobulk_matrix <- read.table("fig5_region_donor_ID_pseudobulk.csv", header = TRUE, sep=",",row.names=1
dat=log10(edgeR::cpm(pseudobulk_matrix)+1)
# transpose
t_exp <- t(dat)
# convert to data.frame
t_exp <- as.data.frame(t_exp)
# choose target gene
target_gene <- 'TNF'

# Calculate the correlation coefficient and p value between each gene and the target gene
cor_list <- list()

for (i in colnames(t_exp)) {
  # Extract the expression value of the target gene
  tar <- t_exp[,target_gene]
  # Pearson correlation test
  cor_res <- cor(x = tar,y = t_exp[,i],method = 'pearson')
  # pvalue
  cor_pval <- cor.test(x = tar,y = t_exp[,i])$p.value
  # merge results
  final_res <- data.frame(tar_genename = target_gene,
                          gene_name = i,
                          cor_results = cor_res,
                          cor_pvalue = cor_pval)
  # save
  cor_list[[i]] <- final_res
}

# merge results
gene_corres <- do.call('rbind',cor_list)

head(gene_corres,4)
write.csv(gene_corres,file="TNF_allgene_cor.csv")
### Get a subset matrix of correlation coefficients between TNF and glutamate receptors

# Specify gene list
gene_list <- c(
    "GRIA1", "GRIA2", "GRIA3", "GRIA4", "GRID1", "GRID2", 
    "GRIK1", "GRIK2", "GRIK3", "GRIK4", "GRIK5", "GRIN1", 
    "GRIN2A", "GRIN2B", "GRIN2C", "GRIN2D", "GRIN3A", "GRIN3B",
    "GRINA", "GRM1", "GRM2", "GRM3", "GRM4", "GRM5", "GRM6",
    "GRM7", "GRM8"
)

# Select a row name in the gene list to create a subset
nr_gene_corres <- gene_corres[gene_corres$gene %in% gene_list, ]

nr_gene_corres$change <- ifelse(nr_gene_corres$cor_results > 0, "up", "down")


nr_gene_corres$'-log10pvalue' <- ifelse(
  nr_gene_corres$change == 'up', -log10(nr_gene_corres$cor_pvalue), -(-log10(nr_gene_corres$cor_pvalue))
  )
head(nr_gene_corres)

gene_order <- match(rownames(nr_gene_corres), gene_list)

nr_gene_corres <- nr_gene_corres[gene_order, ]

write.csv(nr_gene_corres,file="TNF_nrgene_corres.csv")
p4 <- ggplot(nr_gene_corres, aes(x = `-log10pvalue`,y = gene_name)) +
  geom_col(aes(fill = `-log10pvalue`), width = 0.1) +
  geom_point(aes(size = cor_results,
           color = `-log10pvalue`)) +
  scale_size_continuous(range = c(2, 7)) +
  scale_color_continuous_c4a_div('sunset', mid = 0, reverse = T) +
  scale_fill_continuous_c4a_div('sunset', mid = 0, reverse = T) +
  theme_classic()
p4

ggsave(filename="extended_data_fig6c_left.pdf",plot=p4, width = 5, height = 10)

## IL1A

In [ ]:
library(tidyverse)
library(ggplot2)
library(ggpubr)
library(cowplot)
library(ggExtra)
library(ggplot2)
library(cols4all)

pseudobulk_matrix <- read.csv("fig5_region_donor_ID_pseudobulk.csv", header = TRUE, sep=",",row.names=1)

dat=log10(edgeR::cpm(pseudobulk_matrix)+1)


t_exp <- t(dat)

t_exp <- as.data.frame(t_exp)

target_gene <- 'IL1A'

cor_list <- list()
for (i in colnames(t_exp)) {

  tar <- t_exp[,target_gene]

  cor_res <- cor(x = tar,y = t_exp[,i],method = 'pearson')

  cor_pval <- cor.test(x = tar,y = t_exp[,i])$p.value

  final_res <- data.frame(tar_genename = target_gene,
                          gene_name = i,
                          cor_results = cor_res,
                          cor_pvalue = cor_pval)

  cor_list[[i]] <- final_res
}


gene_corres <- do.call('rbind',cor_list)

head(gene_corres,4)
write.csv(gene_corres,file="IL1A_allgene_corres.csv")


# specify gene list
gene_list <- c(
    "GRIA1", "GRIA2", "GRIA3", "GRIA4", "GRID1", "GRID2", 
    "GRIK1", "GRIK2", "GRIK3", "GRIK4", "GRIK5", "GRIN1", 
    "GRIN2A", "GRIN2B", "GRIN2C", "GRIN2D", "GRIN3A", "GRIN3B",
    "GRINA", "GRM1", "GRM2", "GRM3", "GRM4", "GRM5", "GRM6",
    "GRM7", "GRM8"
)


# Select a row name in the gene list to create a subset
nr_gene_corres <- gene_corres[gene_corres$gene %in% gene_list, ]

nr_gene_corres$change <- ifelse(nr_gene_corres$cor_results > 0, "up", "down")


nr_gene_corres$'-log10pvalue' <- ifelse(
  nr_gene_corres$change == 'up', -log10(nr_gene_corres$cor_pvalue), -(-log10(nr_gene_corres$cor_pvalue))
  )
head(nr_gene_corres)

gene_order <- match(rownames(nr_gene_corres), gene_list)

nr_gene_corres <- nr_gene_corres[gene_order, ]
write.csv(nr_gene_corres,file="IL1A_nrgene_corres.csv")
p5 <- ggplot(nr_gene_corres, aes(x = `-log10pvalue`,y = gene_name)) +
  geom_col(aes(fill = `-log10pvalue`), width = 0.1) +
  geom_point(aes(size = cor_results,
           color = `-log10pvalue`)) +
  scale_size_continuous(range = c(2, 7)) +
  scale_color_continuous_c4a_div('army_rose', mid = 0, reverse = T) +
  scale_fill_continuous_c4a_div('army_rose', mid = 0, reverse = T) +
  theme_classic()
p5

ggsave(filename="extended_data_fig6c_right.pdf",plot=p5, width = 5, height = 10)

p4+p5